# Export onnx with your trained weights

In [2]:
# revise the code according to your model
# the second line is the location of exporting script, base on your nature of your task (classification/ detection/ segmentation)
# the third line is the location of configs of your model
# the fourth line is the location of weights (pth) of your model
# the fifth line is a sample of image
# the sixth line is exporting location
# the seventh line is choosing model to run on cpu or cuda

In [3]:
import os
from glob import glob

def get_latest_checkpoint(base_dir="mmpretrain/work_dirs"):
    subdirs = [os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
    subdirs.sort(key=os.path.getmtime, reverse=True)  # Sort by modification time, newest first

    for d in subdirs:
        ckpt_file = os.path.join(d, "last_checkpoint")
        if os.path.exists(ckpt_file):
            with open(ckpt_file, "r") as f:
                relative_ckpt_path = f.readline().strip()
                full_ckpt_path = os.path.join(d, relative_ckpt_path)
                if os.path.exists(full_ckpt_path):
                    return full_ckpt_path
    return None

latest_ckpt = get_latest_checkpoint()
print(latest_ckpt)

In [5]:
%run mmdeploy/tools/deploy.py \
mmdeploy/configs/mmpretrain/classification_onnxruntime_dynamic.py \
mmpretrain/work_dirs/swinv2-tiny-w8_16xb64_in1k-256px_place/swinv2-tiny-w8_16xb64_in1k-256px_place.py \
{latest_ckpt} \
mmpretrain/data/places365/val/ballroom/Places365_val_00001732.jpg \
--work-dir mmdeploy_model/swinv2 \
--device cuda

05/29 17:30:52 - mmengine - INFO - Start pipeline mmdeploy.apis.pytorch2onnx.torch2onnx in subprocess
05/29 17:30:52 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
05/29 17:30:52 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "mmpretrain_tasks" registry tree. As a workaround, the current "mmpretrain_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
05/29 17:30:52 - mmengine - INFO - Because batch augmentations are enabled, the data preprocessor automatically enables the `to_onehot` op

/home/z890/Downloads/ml_sample/classification/mm_Swinv2/mmcv/mmcv/cnn/bricks/transformer.py:125: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  output_h = math.ceil(input_h / stride_h)
/home/z890/Downloads/ml_sample/classification/mm_Swinv2/mmcv/mmcv/cnn/bricks/transformer.py:126: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  output_w = math.ceil(input_w / stride_w)
/home/z890/Downloads/ml_sample/classification/mm_Swinv2/mmcv/mmcv/cnn/bricks/transformer.py:127: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We 

05/29 17:30:54 - mmengine - INFO - Execute onnx optimize passes.
05/29 17:30:55 - mmengine - INFO - Finish pipeline mmdeploy.apis.pytorch2onnx.torch2onnx
05/29 17:30:55 - mmengine - INFO - Start pipeline mmdeploy.apis.utils.utils.to_backend in main process
05/29 17:30:55 - mmengine - INFO - Finish pipeline mmdeploy.apis.utils.utils.to_backend
05/29 17:30:55 - mmengine - INFO - visualize onnxruntime model start.
05/29 17:30:57 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
05/29 17:30:57 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "mmpretrain_tasks" registry tree. As a workaround, the current "mmpretrain_tasks" registry in "mmdeploy" is used to b

2025-05-29 17:30:57.726813864 [W:onnxruntime:, session_state.cc:1280 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-05-29 17:30:57.726830459 [W:onnxruntime:, session_state.cc:1282 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


05/29 17:30:57 - mmengine - WARNING - expect input device cuda but get cuda:0.
05/29 17:31:31 - mmengine - INFO - visualize onnxruntime model success.
05/29 17:31:31 - mmengine - INFO - visualize pytorch model start.
05/29 17:31:33 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
05/29 17:31:33 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "mmpretrain_tasks" registry tree. As a workaround, the current "mmpretrain_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
05/29 17:31:33 - mmengi

# Inference the onnx

In [6]:
# providers set provider priority cuda or cpu
import onnxruntime
sess = onnxruntime.InferenceSession("mmdeploy_model/swinv2/end2end.onnx", providers=["CUDAExecutionProvider"])
print(sess.get_providers())

['CUDAExecutionProvider', 'CPUExecutionProvider']


2025-05-29 17:31:45.247775561 [W:onnxruntime:, session_state.cc:1280 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-05-29 17:31:45.247789852 [W:onnxruntime:, session_state.cc:1282 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [7]:
'''
from mmdeploy.apis import inference_model
is the official method to run an onnx, but it require torch
'''
#Classification
from mmdeploy.apis import inference_model
result = inference_model(
    model_cfg='mmpretrain/work_dirs/swinv2-tiny-w8_16xb64_in1k-256px_place/swinv2-tiny-w8_16xb64_in1k-256px_place.py',
    deploy_cfg='mmdeploy/configs/mmpretrain/classification_onnxruntime_dynamic.py',
    backend_files=['mmdeploy_model/swinv2/end2end.onnx'],
    img='mmpretrain/data/places365/val/ballroom/Places365_val_00001732.jpg',
    device='cuda')
print(result)

05/29 17:31:52 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
05/29 17:31:52 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "mmpretrain_tasks" registry tree. As a workaround, the current "mmpretrain_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
05/29 17:31:52 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "backend_classifiers" registry tree. As a workaround, the current "backend_classifiers" registry in "mmdeploy" is used to build instance. This ma

2025-05-29 17:31:52.298685018 [W:onnxruntime:, session_state.cc:1280 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-05-29 17:31:52.298698078 [W:onnxruntime:, session_state.cc:1282 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


[<DataSample(

META INFORMATION
    img_shape: (256, 256)
    ori_shape: (256, 256)
    img_path: mmpretrain/data/places365/val/ballroom/Places365_val_00001732.jpg
    scale_factor: (1.140625, 1.140625)
    num_classes: 28

DATA FIELDS
    pred_score: tensor([0.0058, 0.0167, 0.0689, 0.0767, 0.0246, 0.0134, 0.0311, 0.0079, 0.0382,
        0.1331, 0.0139, 0.0109, 0.0169, 0.0136, 0.0576, 0.0059, 0.0091, 0.0617,
        0.0495, 0.0139, 0.0088, 0.0251, 0.0286, 0.0405, 0.0296, 0.0816, 0.0765,
        0.0397])
    pred_label: tensor([9])

) at 0x7d58483f4f40>]


In [8]:
'''
this is an unofficial method to run an onnx that require no torch
however, you need to check the 'model_cfg' to understand how the image was preprocessed
which include normalization method and resize and padding method
you may search keywords "data_preprocessor" and "mean" and "std" for normalization 
and "pipeline" and "scale" for resize and padding method
'''
#Classification 
import onnxruntime
import numpy as np
from PIL import Image

def resize_edge(img, scale=292, edge='short', interpolation='bicubic'):
    """Resize so that the short edge is `scale`, maintaining aspect ratio."""
    width, height = img.size
    if edge == 'short':
        if width < height:
            new_width = scale
            new_height = int(scale * height / width)
        else:
            new_height = scale
            new_width = int(scale * width / height)
    elif edge == 'long':
        if width > height:
            new_width = scale
            new_height = int(scale * height / width)
        else:
            new_height = scale
            new_width = int(scale * width / height)
    else:
        raise ValueError("edge must be 'short' or 'long'")

    resample = Image.BICUBIC if interpolation == 'bicubic' else Image.BILINEAR
    return img.resize((new_width, new_height), resample)

def center_crop(img, crop_size=256):
    """Crop the center `crop_size` x `crop_size` region from the image."""
    width, height = img.size
    left = (width - crop_size) // 2
    top = (height - crop_size) // 2
    right = left + crop_size
    bottom = top + crop_size
    return img.crop((left, top, right, bottom))

    
# Load the ONNX model with CUDA execution provider
sess = onnxruntime.InferenceSession("mmdeploy_model/swinv2/end2end.onnx", providers=["CPUExecutionProvider"])
print("Available providers:", sess.get_providers())

# Define the image path
image_path = "mmpretrain/data/places365/val/ballroom/Places365_val_00001732.jpg"

# Load and preprocess the image
image = Image.open(image_path)
'''
according to config
    test_pipeline = [
        dict(type='LoadImageFromFile'),
        dict(
            backend='pillow',
            edge='short',
            interpolation='bicubic',
            scale=292,
            type='ResizeEdge'),
        dict(crop_size=256, type='CenterCrop'),
        dict(type='PackInputs'),
    ]
'''
image = resize_edge(image, scale=292, edge='short', interpolation='bicubic')
image = center_crop(image, crop_size=256)

# Convert the input tensor to a numpy array (ONNXRuntime uses numpy arrays)
input_array = np.transpose(image, (2,0,1))
input_array = np.expand_dims(input_array,axis=0)

'''
according to config
    data_preprocessor = dict(
        mean=[
            123.675,
            116.28,
            103.53,
        ],
        num_classes=28,
        std=[
            58.395,
            57.12,
            57.375,
        ],
        to_rgb=True)
'''
#normalize image
mean = np.array([123.675, 116.28, 103.53]).reshape(1, 3, 1, 1) 
std = np.array([58.395, 57.12, 57.375]).reshape(1, 3, 1, 1) 
input_array = (input_array - mean) / std
input_array = input_array.astype('float32')

# Get the model's input name (usually 'input' or something similar)
input_name = sess.get_inputs()[0].name

# Run inference on the input image
outputs = sess.run(None, {input_name: input_array})

# Get the output (typically the class probabilities)
output = outputs[0]

# Print the output
print("Model output:", output)


Available providers: ['CPUExecutionProvider']
Model output: [[0.00576688 0.01667846 0.06892081 0.07669645 0.02459264 0.0134371
  0.03105985 0.00792458 0.03824529 0.1331148  0.0139368  0.0109138
  0.01692919 0.01361539 0.05759621 0.00593901 0.0091106  0.0616653
  0.04947931 0.01392222 0.00882916 0.02504956 0.02863899 0.0404804
  0.02963363 0.08163732 0.07646955 0.03971672]]
